In [164]:
import pandas as pd
import numpy as np
import pickle
import os
from lxml import etree
from logos_tools import *
from logos_opta import *

In [165]:
GAME_FILE = '../resources/games_friendly/17_PSG_vs_Nice.xml'

In [166]:
# 读取比赛
xml = etree.parse(GAME_FILE)

In [167]:
# 1.统计时间
# 2.球员交易

In [168]:
# 获取首发
set_up = xml.xpath('//Event[@type="Team set up"]')
def get_from_setup(set_up, qid):
    return sum([[x.strip() for x in event.xpath('Q[@qualifier_id="%s"]' % qid)[0].get('value').split(',')] for event in set_up], [])
position_list = get_from_setup(set_up, '44')
player_list = get_from_setup(set_up, '30')
player_df = pd.DataFrame({'player':player_list, 'position':position_list})

In [169]:
# 结束时间
def get_time(minute, sec):
    return int(minute)*60 + int(sec)

total_time = max([get_time(event.get('min'), event.get('sec')) for event in xml.xpath('//Event[@type="End"]')])
# 简化处理，简单考虑中场的伤停补时
player_df.loc[player_df.position!='5', 'start'] = 0
player_df.loc[:, 'end'] = total_time

In [170]:
# 球员上场，获取开始时间
player_on = xml.xpath('//Event[@type="Player on"]')
player_on_list = [(event.get('player_id'), get_time(event.get('min'), event.get('sec'))) for event in player_on]
for item in player_on_list:
    player_df.loc[player_df.player==item[0], 'start'] = item[1]

In [171]:
# 球员下场，获取结束时间
player_off = xml.xpath('//Event[@type="Player Off"]')
player_off_list = [(event.get('player_id'), get_time(event.get('min'), event.get('sec'))) for event in player_off]
for item in player_off_list:
    player_df.loc[player_df.player==item[0], 'end'] = item[1]

In [172]:
def get_time(minute, sec):
        return int(minute)*60 + int(sec)

In [173]:
def get_player_time_df(xml):
    # 获取首发
    set_up = xml.xpath('//Event[@type="Team set up"]')
    def get_from_setup(set_up, qid):
        return sum([[x.strip() for x in event.xpath('Q[@qualifier_id="%s"]' % qid)[0].get('value').split(',')] for event in set_up], [])
    position_list = get_from_setup(set_up, '44')
    player_list = get_from_setup(set_up, '30')
    player_df = pd.DataFrame({'player':player_list, 'position':position_list})
    # 结束时间
    total_time = max([get_time(event.get('min'), event.get('sec')) for event in xml.xpath('//Event[@type="End"]')])
    # 简化处理，简单考虑中场的伤停补时
    player_df.loc[player_df.position!='5', 'start'] = 0
    player_df.loc[:, 'end'] = total_time
    # 球员上场，获取开始时间
    player_on = xml.xpath('//Event[@type="Player on"]')
    player_on_list = [(event.get('player_id'), get_time(event.get('min'), event.get('sec'))) for event in player_on]
    for item in player_on_list:
        player_df.loc[player_df.player==item[0], 'start'] = item[1]
    # 球员下场，获取结束时间
    player_off = xml.xpath('//Event[@type="Player Off"]')
    player_off_list = [(event.get('player_id'), get_time(event.get('min'), event.get('sec'))) for event in player_off]
    for item in player_off_list:
        player_df.loc[player_df.player==item[0], 'end'] = item[1]
    player_df = player_df.loc[player_df.start.notna()].copy()
    player_df.loc[:,'time'] = (player_df.end - player_df.start)
    return player_df

In [174]:
# 获取所有的比赛并获得球员总时间
game_dir = '../resources/games_friendly/'
all_games = [f for f in os.listdir(game_dir) if os.path.isfile(os.path.join(game_dir, f))]
player_time_list = [get_player_time_df(etree.parse(os.path.join(game_dir,x))) for x in all_games]

In [175]:
player_time_df = pd.concat(player_time_list)
player_s = player_time_df.groupby('player')['time'].sum()

In [176]:
# 每个人增加2*19分钟，放宽限制（上半场伤停补时） 不使用该方案

In [177]:
player_s = player_s.loc[player_s>(800*60)]

In [178]:
len(player_s)

235

In [99]:
# 转会
PLAYER_FILE = '../resources/Players and IDs - F40 - L1 20162017.xml'
xml = etree.parse(PLAYER_FILE)
playerChanges = xml.xpath('//PlayerChanges')[0]
playerChanges = playerChanges.xpath('Team/Player')
playerid = []
leave_date = []
for x in playerChanges:
    playerid.append(x.get('uID')[1:])
    leave_date.append(x.xpath('Stat[@Type="leave_date"]')[0].text)
playerChanges_df = pd.DataFrame({'player':playerid, 'leave_date':leave_date})

In [100]:
# 有交易记录的球员全部去除
playerChanges_df.loc[playerChanges_df.leave_date>'2017-02-01']

In [136]:
# 去除被交易的球员
player_s = player_s.loc[list(set(list(player_s.index)) - set(list(playerChanges_df.player)))]

In [141]:
# 保存有意义的球员
use_player_file = '../resources/use_player'
with open(use_player_file, 'wb') as handle:
    pickle.dump(set(player_s.index), handle, protocol=pickle.HIGHEST_PROTOCOL)

In [142]:
# with open(use_player_file, 'rb') as handle:
#     test=pickle.load(handle)

In [147]:
def decide_side(x):
    if x in ('Left','Right','Left/Right'):
        return 'side'
    elif x == 'Centre':
        return 'centre'
    else:
        return 'multi'

In [159]:
# 获取球员信息
dim_player_file = '../resources/Players and IDs - F40 - L1 20162017.xml'
xml = etree.parse(dim_player_file)
player_id = []
player_name = []
position = []
real_position = []
real_position_side = []
jersey_num = []
for element in xml.xpath('SoccerDocument/Team/Player'):
    player_id.append(element.get('uID')[1:])
    player_name.append(element.xpath('Name')[0].text)
    position.append(element.xpath('Position')[0].text)
    real_position.append(element.xpath("Stat[@Type='real_position']")[0].text)
    real_position_side.append(element.xpath("Stat[@Type='real_position_side']")[0].text)
    jersey_num.append(element.xpath("Stat[@Type='jersey_num']")[0].text)
all_player_df = pd.DataFrame({'player_id':player_id, 'player_name':player_name, 'position':position, 
                              'real_position':real_position, 'real_position_side':real_position_side, 
                              'jersey_num':jersey_num}, index=player_id)
all_player_df['position_use'] = all_player_df.position + '-' + all_player_df.real_position_side.map(decide_side)

In [160]:
# 去重
all_player_df = all_player_df.drop_duplicates(subset='player_id', keep='last')
file = '../resources/dim/players.tsv'
all_player_df.to_csv(file, sep='\t', index=False)